In [37]:
import pandas as pd

In [3]:
df=pd.read_csv('train.csv')

In [4]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [5]:
df.text[1]

'Ever get the feeling your life circles the roundabout rather than heads in a straight line toward the intended destination? [Hillary Clinton remains the big woman on campus in leafy, liberal Wellesley, Massachusetts. Everywhere else votes her most likely to don her inauguration dress for the remainder of her days the way Miss Havisham forever wore that wedding dress.  Speaking of Great Expectations, Hillary Rodham overflowed with them 48 years ago when she first addressed a Wellesley graduating class. The president of the college informed those gathered in 1969 that the students needed “no debate so far as I could ascertain as to who their spokesman was to be” (kind of the like the Democratic primaries in 2016 minus the   terms unknown then even at a Seven Sisters school). “I am very glad that Miss Adams made it clear that what I am speaking for today is all of us —  the 400 of us,” Miss Rodham told her classmates. After appointing herself Edger Bergen to the Charlie McCarthys and Mor

In [6]:
df.shape

(20800, 5)

In [7]:
df.dropna(inplace=True)

In [8]:
df.shape

(18285, 5)

In [9]:
data=df.reset_index()

In [10]:
import spacy
nlp=spacy.load("en_core_web_md")

In [11]:
corpus=data["text"].apply(lambda text: " ".join(token.lemma_ for token in nlp(text) if not token.is_punct and not token.is_stop and not token.is_currency
              and not token.is_digit
              and not token.is_oov
              and not token.is_space
              and not token.like_num
              and not token.pos_ == "PROPN"))

In [12]:
corpus=corpus.apply(lambda x : " ".join(y.lower() for y in x.split()))

In [13]:
corpus

0        letter tweet stump image courtesy available li...
1        feeling life circle roundabout head straight l...
2        fire tension intelligence analyst political po...
3        kill identify rate civilian kill american airs...
4        print iranian woman sentence year prison searc...
                               ...                        
18280    unload black celebrity meet election say fail ...
18281    lose drop vow run table march playoff victory ...
18282    today grow union great name american retailing...
18283    hold tv military force hold parallel military ...
18284    author activist journalist radio host director...
Name: text, Length: 18285, dtype: object

In [14]:
import re
corpus=corpus.apply(lambda x: re.sub('[^A-Za-z ]',' ',x))

In [15]:
corpus[0]

'letter tweet stump image courtesy available license apology doubt week accord democratic aide look like know bad person turn send infamous letter announce look email relate email server rank relevant committee hear find tweet republican committee chairman know notify republican chairman democratic rank member committee agency review email recently discover order contain classified information long letter go set political world ablaze tweet inform learn existence email appear pertinent investigation case reopen course know case actually say review email light unrelated know sexte teenager apparently little thing fact matter vow initiate raft investigation win year worth possibly entire term worth apparently think work result tweet briefly roil nation cooler head realize dud accord senior democratic aide misread letter chaffetz sin aide tell boss know letter time find check relevant committee receive letter fact didn receive tweet public let get right director tell committee chairman ma

###### BOW

In [16]:
from sklearn.feature_extraction.text import CountVectorizer
countvec=CountVectorizer(max_features=5000,ngram_range=(1,3))

In [17]:
X=countvec.fit_transform(corpus).toarray()

In [18]:
countvec.get_feature_names_out()[:50]

array(['abandon', 'ability', 'able', 'aboard', 'abortion', 'abroad',
       'absence', 'absolute', 'absolutely', 'absorb', 'absurd', 'abuse',
       'academic', 'academy', 'accelerate', 'accept', 'acceptable',
       'acceptance', 'access', 'accessible', 'accident', 'accommodate',
       'accompany', 'accomplish', 'accomplishment', 'accord',
       'accord people', 'accord report', 'account', 'accountability',
       'accountable', 'accumulate', 'accurate', 'accurately',
       'accusation', 'accuse', 'achieve', 'achievement', 'acid',
       'acknowledge', 'acquire', 'acquisition', 'acre', 'act', 'act like',
       'action', 'activate', 'active', 'actively', 'activism'],
      dtype=object)

In [19]:
X.shape

(18285, 5000)

In [20]:
y=data['label']

In [21]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3)

In [22]:
count_df=pd.DataFrame(X_train,columns=countvec.get_feature_names_out())

In [23]:
count_df.head()

,abandon,ability,able,aboard,abortion,abroad,absence,absolute,absolutely,absorb,...,yield,young,young child,young man,young people,young woman,youth,zika,zionist,zone
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,1,0,1,0,0,1,0,0,0
3,0,0,1,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,3,0,0,0,0,0,0,0,0


In [24]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

In [25]:
mnb=MultinomialNB()
mnb.fit(X_train,y_train)

MultinomialNB()

In [26]:
mnb_pred_test=mnb.predict(X_test)
mnb_pred_train=mnb.predict(X_train)

In [27]:
test_score=accuracy_score(mnb_pred_test,y_test)
train_score=accuracy_score(mnb_pred_train,y_train)
print("test score : ",test_score)
print("train score : ",train_score)

test score :  0.8496172074371127
train score :  0.867020861004766


###### TF-IDF


In [28]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vector=TfidfVectorizer(max_features=5000,ngram_range=(1,3))

In [29]:
X_tfidf=tfidf_vector.fit_transform(corpus).toarray()

In [30]:
X_train_tf,X_test_tf,y_train_tf,y_test_tf = train_test_split(X_tfidf,y,test_size=0.3)

In [43]:
X_test_tf[0][0:25]

array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.04001486, 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ])

In [31]:
mnb1=MultinomialNB()
mnb1.fit(X_train_tf,y_train_tf)

MultinomialNB()

In [32]:
tf_pred=mnb1.predict(X_test_tf)

In [33]:
len(tf_pred)

5486

In [34]:
len(y_test)

5486

In [35]:
accuracy_score(tf_pred,y_test)

0.5231498359460445